In [11]:
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker
from collections import Counter
from nltk.corpus import stopwords
import nltk
import re

In [12]:
def read_msgs_file(friend: str, username: str):
    file_path = friend + '.txt'
    ref = []
    printable_chars = set(string.printable)
    LTR_MARK = "\u200E"


    with open(file_path, 'r', encoding="utf-8") as file:
        # for line in file:
        for line in file:
            try:
                if "�" in line or LTR_MARK in line:  # replacement character used when decode fails
                    continue
                
                # Skip if any non-printable chars
                if not all(c in printable_chars or c.isspace() for c in line):
                    continue

                line = line.split('] ')
                date, time = map(str, line[0].split(', '))
                date = date[1:]
                if date[-2:] != '25': continue

                line = line[1].split(': ')
                frm = line[0]
                msg = line[1][:-1]
                

                ref.append({
                    'date': date,
                    'time': time,
                    'frm': frm,
                    'msg': msg
                })

            except Exception as e:
                continue
            


    df = pd.DataFrame(ref)
    df['frm'] = np.where(df['frm'] == username, 'me',
           np.where(df['frm'] != username, 'friend', df['frm']))
    return df


In [13]:
df = read_msgs_file(friend='zm', username='yf')
df.head()

,date,time,frm,msg
0,1/1/25,11:32:05,me,last seen td 4am is crazy
1,1/1/25,11:32:13,friend,Woooo
2,1/1/25,11:32:21,friend,What laptop
3,1/1/25,11:32:22,friend,Isit
4,1/1/25,11:49:53,me,o is like thinkpad x13


In [14]:
new_ref = list(df.frm + ': ' + df.msg)

In [15]:
with open('cred.json', 'r') as f:
    config = json.load(f)
    
api_key = config['apiKey']
query = "Explain quantum computing in simple terms"
print(api_key)
query

NameError: name 'json' is not defined

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=api_key,
)

completion = client.chat.completions.create(
  extra_body={},
  model="deepseek/deepseek-r1:free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)
print(completion.choices[0].message.content)

ModuleNotFoundError: No module named 'openai'

In [ ]:
base_prompt= '''
Role & Task:
You are a master of monotonous, long-winded, and deeply mundane storytelling. Your task is to write a script for a 5-minute audio segment (approx. 600-750 words) for a hackathon project. The project is a podcast/app designed to help users fall asleep through incredibly boring, rant-filled monologues.

Core Instructions:

Tone & Voice: The script's tone and voice must meticulously match the style found in the following transcript of my messages from the last month. Pay close attention to:

Sentence Structure: (e.g., long, rambling sentences with multiple clauses; frequent use of tangents; repetitive points).

Pacing: (e.g., slow, meandering, with no sense of urgency).

Vocabulary & Topics: (e.g., specific words, phrases, and subjects I frequently revert to).

Cadence & Rhythm: (e.g., how ideas are connected—or not connected—and how a thought evolves over several sentences).

Content: The script should be a first-person monologue. It is not a story with a plot, but a rant or a deep dive into a trivial, everyday subject. The goal is to be just engaging enough to focus on, but so tedious that it effortlessly lulls the listener to sleep. Think of it as "audio melatonin." The topic should be something universally relatable but utterly non-stimulating (e.g., the subtle differences between brands of paper towels, the ordeal of organizing a junk drawer, the history of the stapler, the frustration of a slightly wobbly table).

Output Format:

Format the script clearly for audio, specifying tone and pace in parentheses.

Include subtle, non-intrusive sound effect cues (e.g., [SOUND: Distant, gentle keyboard clacking], [SOUND: Soft sigh]) to enhance the somnambulant atmosphere.

Do not use music.

Ensure the script is approximately 5 minutes long when read at a slow, deliberate pace.

Here is the transcript of my messages for you to analyze and emulate:

[PASTE YOUR TRANSCRIPT HERE]

Begin generating the script now.
'''